In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
from sentence_transformers import SentenceTransformer
from model.decoderMLP import decoderMLP, decoderAttention, movieTransformer
from tqdm import tqdm
import openai
import pickle 
import argparse
from typing import List
import logging
import numpy as np
import torch
import json
import time
import torch.optim as optim
import torch
from pprint import pprint as pp
from scipy.sparse import csr_matrix
import os 
import pandas as pd
from collections import defaultdict
from helper.sampler import NegSampler, negsamp_vectorized_bsearch_preverif
from argparse import ArgumentParser
from model.MF import MatrixFactorization,MatrixFactorizationLLM
from trainer.training_utils import *
from helper.eval_metrics import *
from helper.dataloader import *

/home/mila/e/emiliano.penaloza/llm4rec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
lr= 0.00001
epochs = 500
num_heads = 8
cosine = False
num_layers = 2
output_emb = 256
embedding_dim = 768

saved_path = f'/home/mila/e/emiliano.penaloza/scratch/saved_model/ml-100k/attention_best_model_{lr}_{epochs}_{num_heads}_{cosine}_{num_layers}.pth'

model_path = saved_path + '_best_model.pth'
embedder_path = saved_path + '_embedder.pth'
item_embeddings_path = saved_path + '_item_embeddings.pth'
user_embeddings_path = saved_path + '_user_embeddings.pth'
model_rankings_path = saved_path + '_rankings_matrix.npy'
id_genre_map = map_id_to_genre('../data/ml-100k/movies.dat')

# 1. Data Loading & Preprocessing


train_data = load_dataset("../data_preprocessed/ml-100k/data_split/train_set_leave_one.json")
valid_data = load_dataset("../data_preprocessed/ml-100k/data_split/valid_set_leave_one.json")
test_data = load_dataset("../data_preprocessed/ml-100k/data_split/test_set_leave_one.json")
movie_title_to_id = map_title_to_id("../data/ml-100k/movies.dat")
movie_id_to_title= map_id_to_title("../data/ml-100k/movies.dat")

train_data = convert_titles_to_ids(train_data, movie_title_to_id)
valid_data = convert_titles_to_ids(valid_data, movie_title_to_id)
test_data = convert_titles_to_ids(test_data, movie_title_to_id)

train_matrix, actual_list_val, actual_list_test = create_train_matrix_and_actual_lists(train_data, valid_data,
                                                                                        test_data, movie_title_to_id)

full_data  = create_full_data(train_data,valid_data,test_data)
    
data_matrix = create_full_dataset_matrix(full_data,movie_title_to_id)

train_matrix = csr_matrix(train_matrix)  # Convert train_matrix to a CSR matrix


# 2. Model Creation
num_users, num_items = train_matrix.shape

In [42]:
from collections import defaultdict
counts = np.array(data_matrix).sum(axis = 1 )
dict_out = defaultdict( dict)
for i,c  in enumerate(counts): 
    dict_out[i]['title'] = movie_id_to_title[i+1] 
    dict_out[i]['count'] = c

In [49]:
sorted_data_desc = dict(sorted(dict_out.items(), key=lambda x: x[1]['count'], reverse=True))
print(sorted_data_desc)
# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(sorted_data_desc, orient='index')
df.to_csv('../data/popularity.csv',index=[0])

{404: {'title': 'Mission: Impossible (1996)', 'count': 534}, 654: {'title': 'Stand by Me (1986)', 'count': 504}, 12: {'title': 'Mighty Aphrodite (1995)', 'count': 468}, 449: {'title': 'Star Trek V: The Final Frontier (1989)', 'count': 411}, 275: {'title': 'Leaving Las Vegas (1995)', 'count': 382}, 233: {'title': 'Jaws (1975)', 'count': 366}, 302: {'title': "Ulee's Gold (1997)", 'count': 365}, 415: {'title': 'Old Yeller (1957)', 'count': 365}, 536: {'title': 'My Own Private Idaho (1991)', 'count': 363}, 278: {'title': 'Once Upon a Time... When We Were Colored (1995)', 'count': 325}, 392: {'title': 'Mrs. Doubtfire (1993)', 'count': 325}, 180: {'title': 'Return of the Jedi (1983)', 'count': 317}, 428: {'title': 'Day the Earth Stood Still, The (1951)', 'count': 308}, 845: {'title': 'To Gillian on Her 37th Birthday (1996)', 'count': 302}, 6: {'title': 'Twelve Monkeys (1995)', 'count': 300}, 292: {'title': 'Donnie Brasco (1997)', 'count': 300}, 681: {'title': 'I Know What You Did Last Summer

In [34]:
sorted_dict_out = sorted(dict_out, key=lambda x: x['count'])

TypeError: string indices must be integers